In [56]:
# header
import pandas as pd
import csv
import AddIPAFlex
import json
import MatchHeadwords
from GenerateLexDir import literal_eval_col
from IPython.display import display
from uuid import uuid4

# load in df's
flexicon = pd.read_csv('flexicon_NODUPLS.csv', keep_default_na=False)
senses = pd.read_csv('SensesMERGE.csv', index_col='sense_id', keep_default_na=False)

# take things literally
#literal_eval_col(flexicon, 'sense')
literal_eval_col(flexicon, 'note')
literal_eval_col(flexicon, 'other_sources')
literal_eval_col(flexicon, 'these_vars')
literal_eval_col(flexicon, 'variant_of')

literal_eval_col(senses, 'gloss')
literal_eval_col(senses, 'def')
flexicon.head()

,entry_id,bibliography,date,date_modified,headword,morph_type,note,other_sources,pronunciation,sense,these_vars,variant_of
0,wao taah_002ca005-1366-4a7b-aff1-ac10db10b768,Fieldnotes Epps/Obert; 2018,2019-01-14T03:46:55Z,,wao taah,phrase,None,None,waʔoʔ ta:h,['802366cc-fff6-4c35-bff4-a403f4b4130d'],None,{}
1,janooh_022d930d-b619-4a77-8ff4-fa5ea30739e8,Fieldnotes Epps/Obert; 2018,2018-07-07T20:57:29Z,,janooh,stem,{},None,jaˈno:h,['9e409799-d500-44e1-97b4-66925497edde'],None,{}
2,p’aa hẽnh_022f0eb2-2301-40e8-809c-927ccebeff3c,Fieldnotes Epps/Obert; 2018,2018-07-15T20:18:56Z,,p’aa hẽnh,phrase,None,{'Sil Dict 2011': 'pʹaa hẽnh'},paʔaʔ hẽɲ,['3954f8c6-283b-4e9f-a9b1-5e9ea6096fcc'],{'pʹaa hẽnh_988a43b7-7d72-4d87-be1d-c63c761e1d...,{}
3,dajëp_024eb574-aa9a-4bea-adc6-12857406c334,Fieldnotes Epps/Obert; 2018,2018-07-10T00:09:23Z,,dajëp,stem,{},None,daˈjǝp˺,['a8675526-039d-459c-b4c8-c3434645200d'],None,{}
4,tawarẽẽh1_025c1d2a-ce6b-4f4a-89c6-58f7995ce55a,Fieldnotes Epps/Obert; 2018,2018-03-08T07:17:53Z,,tawarẽẽh,stem,{},None,tawaˈɾẽ:ʔ,['edf56ebd-dec5-40c2-8444-09892cd2d358'],None,{}


In [57]:
# enforce consistent formatting in bibliography column
#bib_bois = set(x for x in flexicon['bibliography'])
#bib_bois
new_bibs = []
for bib in flexicon['bibliography']:
    if not bib:
        new_bibs.append(None)
        continue
    bib = bib.lower()
    bib_str = ''
    if 'epps' in bib:
        bib_str = 'Fieldnotes Epps/Obert'
        if '2018' in bib:
            bib_str += ' 2018'
        elif '2019' in bib:
            bib_str += ' 2019'
    elif 'martins' in bib:
        bib_str = 'Martins 2005'
    elif 'barbosa' in bib:
        bib_str = 'Barbosa 2005'
    elif 'weir' in bib:
        bib_str = 'Weir'
        if '1986' in bib:
            bib_str += ' 1986,'
        if '1990' in bib:
            bib_str += ' 1990,'
        if '1994' in bib:
            bib_str += ' 1994'
        elif bib_str.endswith(','):
            bib_str = bib_str[:-1]
    elif 'sil' in bib:
        bib_str = 'SIL 2011'
    else:
        assert False, bib
    new_bibs.append(bib_str)

flexicon.loc[:,'bibliograghy'] = new_bibs

In [58]:
# also, remove bibliography key from notes column wherever present
for n, b in zip(flexicon['note'],flexicon['bibliography']):
    if n and 'bibliography' in n:
        n.pop('bibliography')

flexicon['note']

0                                                    None
1                                                      {}
2                                                    None
3                                                      {}
4                                                      {}
                              ...                        
1114    {'Note': 'Predicted phonemic form from source ...
1115                                                   {}
1116                                                   {}
1117                                                   {}
1118                                                   {}
Name: note, Length: 1119, dtype: object

In [59]:
# prepend * to hypothetical forms
for index, row in flexicon.iterrows():
    note = row['note']
    if note and 'Note' in note and note['Note'].startswith('Predicted phonemic'):
        if 'Epps' in row['bibliography']:
            note.pop('Note')
        else:
            hdwd = flexicon.at[index, 'headword']
            hdwd = '*'+hdwd
            flexicon.at[index, 'headword'] = hdwd
            #display(flexicon.loc[index])

In [60]:
# standardize PoS tags
pos = set( s for s in  senses['pos'] )
for index, pos in zip(senses.index, senses['pos']):
    if '?' in pos:
        continue
    elif pos.startswith('v'):
        senses.at[index, 'pos'] = 'Verb'
    elif pos.startswith('n'):
        senses.at[index, 'pos'] = 'Noun'
    elif pos.startswith('adj'):
        senses.at[index, 'pos'] = 'Adjective'
    elif pos.startswith('con'):
        senses.at[index, 'pos'] = 'Conjunction'
    elif pos.startswith('int'):
        senses.at[index, 'pos'] = 'Interjection'
    elif pos.startswith('adv'):
        senses.at[index, 'pos'] = 'Adverb'
    elif pos.startswith('pos'):
        senses.at[index, 'pos'] = 'Postposition'
    elif pos.startswith('onom'):
        senses.at[index, 'pos'] = 'Onomoatopoeia'
    elif pos.startswith('pre'):
        senses.at[index, 'pos'] = 'Prefix'
    elif pos.startswith('suf'):
        senses.at[index, 'pos'] = 'Suffix'
    elif pos.startswith('par'):
        senses.at[index, 'pos'] = 'Particle'
    else:
        pass#print(pos)
senses['pos']

sense_id
e92169c5-ce3e-4a09-9da4-f6860afd0478        Noun
802366cc-fff6-4c35-bff4-a403f4b4130d        Noun
a0d825d5-a0ae-402d-8201-6805e39cad0b        Noun
2e7ced09-86a7-4ad6-a7a0-e36f9523d65a        Noun
062365d9-1fcd-4828-980d-5fbacad0019b        Noun
                                          ...   
92a34bd9-4de3-49ff-83f0-1371c7533ac8        adv?
6c538c3e-c9e2-4c56-856c-455bef81dc6d        Noun
b72bc68e-c154-487d-9fee-2457f25cba60      Adverb
10d81977-d2bf-4e0b-a65f-4b3bcbecb92e        Noun
5e84c851-3898-4013-a050-c8bff6e1c368    Particle
Name: pos, Length: 1636, dtype: object

In [61]:
# enforce consistent usage of apostrophes
# start by finding all unique non-alphabetic chars in headwords
nonalpha = set()
for hdwd in flexicon['headword']:
    for char in hdwd:
        if not char.isalpha():
            nonalpha.add(char)
nonalpha

{' ',
 '%',
 "'",
 '(',
 ')',
 '*',
 '-',
 '.',
 '/',
 ':',
 '=',
 '?',
 '́',
 '̃',
 '̰',
 '’'}

In [62]:
nonalpha = list(nonalpha)
with open('punct.json', 'w') as f:
    json.dump(nonalpha, f)

In [63]:
# literally only two unique apostrophes, don't need to load a json file for that
for index in flexicon.index:
    hdwd = flexicon.at[index, 'headword']
    if "'" in hdwd:
        flexicon.at[index, 'headword'] = hdwd.replace("'", '\u2019')
flexicon['headword']

0             wao taah
1               janooh
2            p’aa hẽnh
3                dajëp
4             tawarẽẽh
             ...      
1114       *tḭ̃: hẽɲ
1115             tĩĩnh
1116    seeb %OR% seeh
1117    wëëm %OR% wëm 
1118              eỹỹm
Name: headword, Length: 1119, dtype: object

In [64]:
# update guids to reflect headwords
headwords = list(flexicon['headword'])
guids = [id.split(sep='_')[1] if id else None for id in flexicon['entry_id']]
for i, hdwd in enumerate(headwords):
    ct = headwords[:i].count(hdwd)
    if ct:
        hdwd+=str(ct)
        print(hdwd)
    guid = guids[i]
    if not guid:
        guid = str(uuid4())
        assert guid not in guids
    guids[i] = hdwd+'_'+guid

hooj1
yb1
 =kóm1
*hẽ1
*na1
*ʃa:h1
*ko:k1
*cɨ:1
*do:1
*pɔ̰̃:d1
*do:2
*ma1
*puh1
*na2


In [65]:
flexicon.loc[:,'entry_id'] = guids
flexicon = flexicon.set_index('entry_id')

In [66]:
flexicon.to_csv('flexicon_3_3-FORMATTED.csv')
senses.to_csv('senses_3_3-FORMATTED.csv')